In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')


In [ ]:
data_fake.head()

In [ ]:
data_true.head()

In [ ]:
data_fake['class'] = 0
data_true['class'] = 1

In [ ]:
data_fake.shape, data_true.shape

In [ ]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i], axis = 0, inplace = True)

In [ ]:
data_fake.shape, data_true.shape

In [ ]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

In [ ]:
data_fake_manual_testing.head(10)

In [ ]:
data_true_manual_testing.head(10)

In [ ]:
data_merge = pd.concat([data_fake, data_true], axis=0)
data_merge.head(10)

In [ ]:
data_merge.columns

In [ ]:
data = data_merge.drop(['title', 'subject', 'date'], axis=1)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.sample(frac=1)

In [ ]:
data.head()

In [ ]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
def wordopt(text):
    # lowercase
    text = text.lower()

    # remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)

    # remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # remove punctuation
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)

    # remove newlines
    text = re.sub(r'\n', ' ', text)

    # remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    return text

In [ ]:
data['text'] = data['text'].apply(wordopt)

In [ ]:
x = data['text']
y = data['class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizartion = TfidfVectorizer()
xv_train = vectorizartion.fit_transform(x_train)
xv_test = vectorizartion.transform(x_test)



In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_lr))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

In [ ]:
print("Classification Report:")
print(classification_report(y_test, pred_dt))

In [81]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [82]:
predit_gb = GB.predict(xv_test)

In [83]:
GB.score(xv_test, y_test)

0.9947638146167558

In [84]:
print(classification_report(y_test, predit_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4683
           1       0.99      1.00      0.99      4293

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [85]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rf = RF.predict(xv_test)

In [ ]:
RF.score(xv_test, y_test)

In [90]:
print(classification_report(y_test, pred_rf))

ValueError: Found input variables with inconsistent numbers of samples: [8976, 8083]

In [ ]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"


def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)

    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]

    new_xv_test = vectorizartion.transform(new_x_test)

    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    print("\n📰 News Text:\n", news)
    print("\n🔍 Model Predictions:")
    print("Logistic Regression:", output_label(pred_LR[0]))
    print("Decision Tree      :", output_label(pred_DT[0]))
    print("Gradient Boosting  :", output_label(pred_GB[0]))
    print("Random Forest      :", output_label(pred_RF[0]))


In [86]:
news = input()
manual_testing(news)

The Ministry of Technology announced on Monday that all citizens will receive free high-speed internet starting next month. Officials claim the program will be funded by unused satellite bandwidth and will provide unlimited data to every household. Experts say this move will make the country a global leader in digital equality. Citizens will not need to register, as internet access will automatically activate on all devices nationwide.

📰 News Text:
 The Ministry of Technology announced on Monday that all citizens will receive free high-speed internet starting next month. Officials claim the program will be funded by unused satellite bandwidth and will provide unlimited data to every household. Experts say this move will make the country a global leader in digital equality. Citizens will not need to register, as internet access will automatically activate on all devices nationwide.

🔍 Model Predictions:
Logistic Regression: Fake News
Decision Tree      : Fake News
Gradient Boosting  : 